In [3]:
import random
import numpy as np
import crocoddyl
crocoddyl.switchToNumpyArray()
try:
   from six.moves import cPickle
except:
   import pickle as cPickle


def generate_trajectories_random_xy(ntraj:int = 10, maxiter:int = 1000, theta_value:int = 0):


    starting_configurations = []
    optimal_trajectories = []
    complete_trajectory = []
    controls = []
    iterations = []
    for _ in range(ntraj):
        initial_config = [random.uniform(-2.1, 2.1), random.uniform(-2.1, 2.1), theta_value]            
        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1., 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 100, model)
        ddp = crocoddyl.SolverDDP(problem)
        ddp.solve([], [], maxiter)


        if ddp.isFeasible:initial_config = [random.uniform(-2.1, 2.1), random.uniform(-2.1, 2.1), theta_value]            
        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1., 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 100, model)
        ddp = crocoddyl.SolverDDP(problem)
        ddp.solve([], [], maxiter)
            state = np.array(ddp.xs)
            control = np.array(ddp.us)                
            starting_configurations.append(state[0,:])
            optimal_trajectories.append(state[1:,:])
            controls.append(control)
            iterations.append(ddp.iter)
    return starting_configurations, optimal_trajectories, controls, iterations

In [116]:
print("Generating 10k trajectories, theta fixed at 0")
start, states, controls, iterations = generate_trajectories_random_xy(ntraj=10)
print('.......')
iterations.sort()
print(f"Total allowed iterations: {1000}, maximum iterations taken to generate 1 trajectory {iterations[-1]}, minimum {iterations[0]}, average {mean(iterations)} ")

Generating 10k trajectories, theta fixed at 0
.......
Total allowed iterations: 1000, maximum iterations taken to generate 1 trajectory 67, minimum 10, average 41.8 


In [141]:
import matplotlib.pyplot as plt
import numpy as np

x = state[0:,0]

y = state[0:,1]

z = np.asarray(control)

#plt.contourf(x, y, z, 20, cmap='RdGy')
#plt.colorbar()
X, Y = np.meshgrid(x, y)

In [147]:
Y

array([[0.59500045, 0.59500045, 0.59500045, ..., 0.59500045, 0.59500045,
        0.59500045],
       [0.55001576, 0.55001576, 0.55001576, ..., 0.55001576, 0.55001576,
        0.55001576],
       [0.44066349, 0.44066349, 0.44066349, ..., 0.44066349, 0.44066349,
        0.44066349],
       ...,
       [0.05768047, 0.05768047, 0.05768047, ..., 0.05768047, 0.05768047,
        0.05768047],
       [0.05768047, 0.05768047, 0.05768047, ..., 0.05768047, 0.05768047,
        0.05768047],
       [0.05768047, 0.05768047, 0.05768047, ..., 0.05768047, 0.05768047,
        0.05768047]])

In [130]:
len(z)

100

In [148]:
def f(x, y):
    return np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)

x = np.linspace(0, 5, 50)
y = np.linspace(0, 5, 40)

X, Y = np.meshgrid(x, y)
Z = np.meshgrid(z)

In [150]:
Z

[array([ 3.29779480e-02,  2.92061562e+00, -1.56236397e+00,  2.01127283e+00,
        -2.30975275e+00,  9.05142529e-01, -2.38380703e+00, -4.55693033e-02,
        -2.05522943e+00, -7.38770595e-01, -1.51603579e+00, -1.15027506e+00,
        -9.08859062e-01, -1.29319837e+00, -3.48102832e-01, -1.22031707e+00,
         9.10924911e-02, -1.01046769e+00,  3.81350260e-01, -7.41399062e-01,
         5.32600588e-01, -4.71374626e-01,  5.73505453e-01, -2.35738398e-01,
         5.37307839e-01, -5.13982004e-02,  4.54661132e-01,  7.74677541e-02,
         3.50829205e-01,  1.54848999e-01,  2.44808587e-01,  1.89484780e-01,
         1.49328329e-01,  1.92199620e-01,  7.14585268e-02,  1.73861556e-01,
         1.36837895e-02,  1.43997855e-01, -2.47875616e-02,  1.10065596e-01,
        -4.67340433e-02,  7.72686691e-02, -5.58374034e-02,  4.87488359e-02,
        -5.58884921e-02,  2.59780754e-02, -5.02838944e-02,  9.21666685e-03,
        -4.17680498e-02, -2.05216087e-03, -3.23580972e-02, -8.75977314e-03,
        -2.3

In [284]:
from crocoddyl import *
initial_config = [random.uniform(-2.1, 2.1), random.uniform(-2.1, 2.1), 0]            
model = crocoddyl.ActionModelUnicycle()
data  = model.createData() 
model.costWeights = np.matrix([1., 0.3]).T
log = []
problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 100, model)
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([CallbackDDPLogger()])
print("...............")
ddp.solve([], [])
print(".......")


NameError: name 'CallbackDDPLogger' is not defined

In [270]:
data

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [264]:
log

In [287]:
import numpy as np

import crocoddyl

# Creating an action model for the unicycle system
model = crocoddyl.ActionModelUnicycle()

# Setting up the cost weights
model.r = [
    10.,  # state weight
    1.  # control weight
]

# Formulating the optimal control problem
T = 20  # number of knots
x0 = np.matrix([-1.2, -1.1, 1.]).T  #x,y,theta
problem = crocoddyl.ShootingProblem(x0, [model] * T, model)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
cost = []
ddp.setCallbacks([crocoddyl.CallbackVerbose()])
# Solving it with the DDP algorithm
ddp.solve([], [])
for d in ddp.datas():
    cost.append(d.cost)
    # Plotting the solution, solver convergence and unicycle motion



In [289]:
cost    

[252.027174176234,
 45.615980653575264,
 6.248861796322507,
 0.8228473079171914,
 0.11807848045861255,
 0.02748693718407192,
 0.0157512550234342,
 0.01419908993700489,
 0.013987395683030531,
 0.013957474743217355,
 0.013953097794284348,
 0.013952439379237945,
 0.013952338427390482,
 0.01395232278657858,
 0.013952320354875695,
 0.013952319977361755,
 0.013952319919021563,
 0.013952319910063116,
 0.013952319908697502,
 0.013952319908491134,
 0.013952319908461847]

ArgumentError: Python argument types in
    None.__init__(ShootingProblem, list, list, ActionModelUnicycle)
did not match C++ signature:
    __init__(_object*  self, Eigen::Matrix<double, -1, 1, 0, -1, 1>  initialState, std::vector<crocoddyl::ActionModelAbstract*, std::allocator<crocoddyl::ActionModelAbstract*> >  runningModels, crocoddyl::ActionModelAbstract*  terminalModel)

In [ ]:
class _unicycle_analysis:
    